### Non-max Suppression

* NMS
Applied to remove redundant bounding boxes after predictions are made.

Before: all predicted bounding boxes:
<img src="./ref_imgs/nms-before.png" alt="Description" width="100">
 
After: nms to remove redundant bboxes
<img src="./ref_imgs/nms-after.png" alt="Description" width="100">


<b>What NMS is doing.</b>

0. Before we do NMS, we have a basic min_threshold this is not the IOU_threshold, this threshold is just to remove bboxes with a low confidence score. then do the following.  

1. Example: we get 3 bboxes.  
<img src="./ref_imgs/during-nms.png" alt="Description" width="200">
    1. each of them have a probability score associated with that box 0.3, 0.9, 0.6, the probability is bewteen 0-1 and it indicts how likely that there is an object in that bounding box.

2. We grab the highest bbox which in this case is the one with a 0.9 probability.  
<img src="./ref_imgs/nms-01.png" alt="Description" width="200">

3. Then we compare it with the other boxes that have a lower probability, and we calculate the IOU between them.
<img src="./ref_imgs/nms-02.png" alt="Description" width="200">
    1. Example IOU = between 0.9 vs 0.6 bboxes = 0.51
    2. if the result is higher than an IOU_threshold, like 0.5 then we remove the 0.6 bbox. 

4. then we compare the 0.9 and 0.3 bboxes, example IOU result = 0.6, which is higher than out IOU_threshold, so we remove the 0.3 bbox

5. If we have multiple class predictions in the image, example: a car and a horse, then we need to do NMS for each different class separately.



In [ ]:
# TODO add a link to implement NMS